In [ ]:
import os
import sys
import pickle
import matplotlib
import numpy as np
import pandas as pd
# import seaborn as sns

from matplotlib import pyplot as plt
from scipy.spatial.transform import Rotation as R

from argparse import ArgumentParser

from filelock import FileLock
from pprint import pprint
from PIL import Image

matplotlib.use('TkAgg')

curr_dir = os.getcwd()
if curr_dir != '/path_to_your_projects/projects/dynamic_dual_manip/':
    os.chdir('/path_to_your_projects/projects/dynamic_dual_manip/')
print(f"Current working directory is {os.getcwd()}")

from bcm.manipulation_utils import generate_fling_trajectory, generate_quasi_static_trajectory, compute_angle


In [ ]:
# Plot per task
from matplotlib import rcParams

# plt.style.use('seaborn')
plt.style.use('seaborn-white')
# rcParams['figure.figsize'] = 11.7, 8.27
rcParams['figure.figsize'] = 6, 6
rcParams['figure.dpi'] = 200
rcParams['figure.subplot.left'] = 0.1
rcParams['figure.subplot.right'] = 0.99
rcParams['figure.subplot.bottom'] = 0.01
rcParams['figure.subplot.top'] = 0.99
rcParams['axes.grid'] = False
# rcParams['grid.color'] = "#ffffff"
rcParams['grid.color'] = "#000000"
rcParams['grid.linewidth'] = 1.0
rcParams['grid.linestyle'] = ":"
# X ticks
rcParams['xtick.major.bottom'] = True
rcParams['xtick.bottom'] = True
rcParams['xtick.minor.visible'] = True
rcParams['xtick.minor.size'] = 5.5
rcParams['xtick.minor.width'] = 1.0
# Y ticks
rcParams['ytick.major.left'] = True
rcParams['ytick.left'] = True
rcParams['ytick.minor.visible'] = True
rcParams['ytick.minor.size'] = 5.5
rcParams['ytick.minor.width'] = 1.0
rcParams['ytick.major.left'] = True
#ytick.major.size:    3.5     # major tick size in points
#ytick.labelsize:     medium  # font size of the tick labels
rcParams['ytick.labelsize'] = 'large'
rcParams['xtick.labelsize'] = "large"

# Font
# rcParams['font.family'] = "serif"
# rcParams['font.style'] = "normal"
rcParams['text.usetex'] = True
rcParams['ps.fonttype'] = 42

# Lines
rcParams['lines.linewidth'] = 0
rcParams['lines.markersize'] = 6

font_value = 41  # Use 50 for the variance plots
# font_value = 100  # Use 50 for the variance plots

rcParams['font.size'] = font_value
# Set specific values for the font relative to font.size
# ## small, medium, large, x-large, xx-large, larger, or smaller
rcParams['axes.titleweight'] = "bold"
# rcParams['axes.labelsize'] = "x-large"
# rcParams['legend.fontsize'] = "xx-large"
# rcParams['legend.title_fontsize'] = 100


In [36]:
# Create the trajectory
pose_1 = [-0.214830, -0.0427511, 0.9253, 0.22292, -0.1000631, 0.9211]
dt = 0.001

traj_left, traj_right =  generate_fling_trajectory(pose_1[:3], pose_1[3:], dt=dt,
                                                   fling_height=pose_1[2] + 0.06,
                                                   grasp_height=pose_1[2] - 0.51051)

In [4]:
tf1 = 1.1
tf2 = 1.5
tf3 = 1.3
tf4 = 0.5

angle_steps = [-179.24297822, -140, -220, -140.0, -140.0]
euler_angle = [-179.24297822, -2.93434324, 42.65659799]  # Defined as X, Y, Z
        
pitch_1, pitch_2, pitch_3 = compute_angle(angle_steps, tf1, tf2, tf3, dt)

temp_roll_traj = np.concatenate((pitch_1[:-1], pitch_2[:-1], pitch_3[:-1]))

last_length = traj_left.shape[0] - temp_roll_traj.shape[0]

pitch_4 = np.zeros(last_length)
pitch_4[:] = angle_steps[-1]

roll_traj = np.concatenate((temp_roll_traj, pitch_4))


In [5]:
# Transform the quaternion into euler angles
euler_traj = np.zeros((traj_left.shape[0], 3))  # Should be X, Y , Z, W

euler_traj[:, 0] = np.deg2rad(roll_traj)
# for i in range(traj_left.shape[0]):
#     temp_euler = R.from_quat([traj_left[i, 3], traj_left[i, 4], traj_left[i, 5], traj_left[i, 6]])
#     # euler = temp_euler.as_euler('xyz', degrees=True)
#     euler = temp_euler.as_euler('xyz')
#     euler_traj[i, :] = euler

In [ ]:
%matplotlib inline

dt_x = np.arange(traj_left.shape[0])*dt

fig1, ax = plt.subplots(3, 1)
fig1.set_size_inches(20, 10, forward=True)
# ax1.set_title('Basic Plot')
# ax[0, 0].set_xlabel('5 actions')
linewidth=3
axv_linewidth=2
ax[0].plot(dt_x, traj_left[:, 1], linewidth=linewidth)
ax[0].set_ylabel('Y (cm)')

ax[1].plot(dt_x, traj_left[:, 2], linewidth=linewidth)
ax[1].set_ylabel('Z (cm)')

ax[2].plot(dt_x, euler_traj[:, 0], linewidth=linewidth)
ax[2].set_ylabel('')
ax[2].set_ylabel('Roll (rad)')


for i in range(3):
    ax[i].set_xlim(0.0, 4.4)
    # ax[i].set_xticklabels([])
    # ax[i].set_yticklabels([])

    ax[i].axvline(x=1.1, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.1+1.5, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.1+1.5+1.3, color='r', ls=':', linewidth=axv_linewidth)
    # ax[i].axvline(x=1.1+1.5+1.3+0.5, color='r', ls=':')
plt.show()
# plt.savefig('pos_quintic_trajectory.pdf')

In [ ]:
%matplotlib inline

vel_y = np.gradient(traj_left[:, 1], dt)
vel_z = np.gradient(traj_left[:, 2], dt)
vel_roll = np.gradient(euler_traj[:, 0], dt)

ddt_x = np.arange(vel_y.shape[0])*dt
print(f"Shape velocity={vel_y.shape[0]}, dt_x max = {dt_x[-1]}")

fig1, ax = plt.subplots(3, 1)
fig1.set_size_inches(20, 10, forward=True)
# ax1.set_title('Basic Plot')
# ax[0, 0].set_xlabel('5 actions')
linewidth=3
axv_linewidth=2
ax[0].plot(ddt_x, vel_y, linewidth=linewidth)
ax[0].set_ylabel('dY (cm/s)')

ax[1].plot(ddt_x, vel_z, linewidth=linewidth)
ax[1].set_ylabel('dZ (cm/s)')

ax[2].plot(ddt_x, vel_roll, linewidth=linewidth)
ax[2].set_ylabel('')
ax[2].set_ylabel('dRoll (rad/s)')


for i in range(3):
    ax[i].set_xlim(0.0, 4.4)
    # ax[i].set_xticklabels([])
    # ax[i].set_yticklabels([])

    ax[i].axvline(x=1.1, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.1+1.5, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.1+1.5+1.3, color='r', ls=':', linewidth=axv_linewidth)
    # ax[i].axvline(x=1.1+1.5+1.3+0.5, color='r', ls=':')
plt.show()
# plt.savefig('vel_quintic_trajectory.pdf')

In [ ]:
%matplotlib inline

acc_y = np.gradient(vel_y, dt)
acc_z = np.gradient(vel_z, dt)
acc_roll = np.gradient(vel_roll, dt)

dddt_x = np.arange(acc_y.shape[0])*dt
print(f"Shape velocity={acc_y.shape[0]}, dt_x max = {dddt_x[-1]}")

fig1, ax = plt.subplots(3, 1)
fig1.set_size_inches(20, 10, forward=True)
# ax1.set_title('Basic Plot')
# ax[0, 0].set_xlabel('5 actions')
linewidth=3
axv_linewidth=2
ax[0].plot(dddt_x, acc_y, linewidth=linewidth)
ax[0].set_ylabel('ddY (cm/s)')

ax[1].plot(dddt_x, acc_z, linewidth=linewidth)
ax[1].set_ylabel('ddZ (cm/s)')

ax[2].plot(dddt_x, acc_roll, linewidth=linewidth)
ax[2].set_ylabel('')
ax[2].set_ylabel('ddRoll (rad/s)')


for i in range(3):
    ax[i].set_xlim(0.0, 4.4)
    # ax[i].set_xticklabels([])
    # ax[i].set_yticklabels([])

    ax[i].axvline(x=1.1, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.1+1.5, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.1+1.5+1.3, color='r', ls=':', linewidth=axv_linewidth)
    # ax[i].axvline(x=1.1+1.5+1.3+0.5, color='r', ls=':')
plt.show()
# plt.savefig('acc_quintic_trajectory.pdf')

# Generate the quasi-static

In [42]:
traj_left_quasi, traj_right_quasi = generate_quasi_static_trajectory(
            pose_1[:3],
            pose_1[3:],
            dt=dt,
            middle_height=pose_1[2] - 0.05,
            touch_height=pose_1[2] - 0.31051,
            modify_x=False
        )

In [ ]:
%matplotlib inline

dt_x = np.arange(traj_left_quasi.shape[0])*dt

fig1, ax = plt.subplots(3, 1)
fig1.set_size_inches(20, 10, forward=True)
# ax1.set_title('Basic Plot')
# ax[0, 0].set_xlabel('5 actions')
linewidth=3
axv_linewidth=2
ax[0].plot(dt_x, traj_left_quasi[:, 1], linewidth=linewidth)
ax[0].set_ylabel('Y (cm)')
ax[0].set_ylim([0.0, -0.7])
ax[1].plot(dt_x, traj_left_quasi[:, 2], linewidth=linewidth)
ax[1].set_ylabel('Z (cm)')
ax[1].set_ylim([0.3, 1.0])

for i in range(2):
    ax[i].set_xlim(0.0, 14.0)
    # ax[i].set_xticklabels([])
    # ax[i].set_yticklabels([])

    ax[i].axvline(x=1.0, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.0+1.0, color='r', ls=':', linewidth=axv_linewidth)
    ax[i].axvline(x=1.1+1.0+6.0, color='r', ls=':', linewidth=axv_linewidth)
    # ax[i].axvline(x=1.1+1.5+1.3+0.5, color='r', ls=':')
# plt.show()
plt.savefig('quasi_pos_quintic_trajectory.pdf')

# Plot sim and target trajectories

In [8]:
sim_cloud_npy = np.load('/path_to_your_projects/projects/dynamic_dual_manip/scripts/sim_cloud_74.npy')
target_vertices = np.load('/path_to_your_projects/projects/dynamic_dual_manip/scripts/target_cloud_74.npy')

In [47]:
elev = 8
azim = 15

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(sim_cloud_npy[:, 0], sim_cloud_npy[:, 1], sim_cloud_npy[:, 2],
               marker='*', color='blue')#, label='Sim')

plt.legend()
ax.view_init(elev=elev, azim=azim)
ax.set_xlim([-0.4, 0.4])
ax.set_ylim([00.0, 1.])
ax.set_zlim([0.00, 1.1])
ax.set_xticklabels([])
ax.set_yticklabels([])
plt.axis('off')
plt.grid(False)
plt.savefig('sim_cloud.png')
# plt.show()
plt.close()

In [ ]:
fig = plt.figure()#dpi=300)
ax = fig.add_subplot(projection='3d')
ax.scatter(target_vertices[:, 0], target_vertices[:, 1], target_vertices[:, 2],
               marker='*', color='k', s=0.8)#, label='Sim')
plt.legend()
ax.view_init(elev=elev, azim=azim)
ax.set_xlim([-0.5, 0.4])
ax.set_ylim([0.0, 1.])
ax.set_zlim([0.00, 1.1])
ax.set_xticklabels([])
ax.set_yticklabels([])
plt.axis('off')
plt.grid(False)
plt.savefig('target_cloud.png')
# plt.show()
plt.close()

# Evaluate all results

In [2]:
import pandas as pd

In [25]:
sim_folders = ['mujoco', 'bullet', 'softgym', 'sofa']
rags = ['cotton', 'chequered', 'linen']
primitives = ['dynamic', 'quasi_static']

all_sim_results = {}
for sim in sim_folders:
    all_sim_results[sim] = {}
    for rag in rags:
        all_sim_results[sim][rag] = {}
        # for primitive in primitives:
        #     all_sim_results[sim][rag][primitive] = {}
            
#     
# all_sim_results = {
#     'mujoco': {
#         'towel': {},
#         'chequered': {},
#         'white': {},
#     },
#     'bullet': {
#         'towel': {},
#         'chequered': {},
#         'white': {},
#     },
#     'softgym': {
#         'towel': {},
#         'chequered': {},
#         'white': {},
#     },
#     'sofa': {
#         'towel': {},
#         'chequered': {},
#         'white': {},
#     },
# }

# results_path = "/path_to_your_projects/projects/dynamic_dual_manip/RESULTS/13_sept_results/"

results_path = "/path_to_your_projects/projects/dynamic_dual_manip/results_27/"


# for result in sim_folders:
for fd in os.listdir(results_path):
    if 'mujoco' in fd:
        env_name = 'mujoco'
    elif 'bullet' in fd:
        env_name = 'bullet'
    elif 'softgym' in fd:
        env_name = 'softgym'
    else:
        env_name = 'sofa'
    

    if 'cotton' in fd:
        fabric = 'cotton'
    elif 'linen' in fd:
        fabric = 'linen'
    else:
        fabric = 'chequered'
    
    if 'quasi_static' in fd:
        primitive  = 'quasi_static'
    else:
        primitive = 'dynamic'
    
    # result = np.load(f'{results_path}/{fd}')
    # print(f"Reading={results_path}/{fd}")
    result = pd.read_csv(f'{results_path}/{fd}')
    if primitive in  all_sim_results[env_name][fabric]:  # Concatenate it
        # print(f"New fabric = {fd} result type={result_type}")
        # print(f"{env_name} {fabric} {result_type} len = {len(sim_results[env_name][fabric][result_type])}")
        all_sim_results[env_name][fabric][primitive] = pd.concat((all_sim_results[env_name][fabric][primitive], result), axis=0)
        # sim_results[env_name][fabric][result_type].merge(result)
    else:  # Create a new one
        # print(f"New fabric = {fd}")
        all_sim_results[env_name][fabric][primitive] = result
        
        

In [ ]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)
nr_decimals=3

result_str = ''
result_values_str = ''
for key, val in all_sim_results.items():
    # print(f'\n===============================')
    result_str += f'\n{key} '
    print(f"Sim={key}")
    for fabric_type, result_values in val.items():
        chamfer_l1_fabric_values = result_values["fabric"]['chamfer_l1_cost_fabric_ls'].values
        chamfer_l1_contact_values = result_values["contact"]['chamfer_l1_cost_contact_ls'].values
        oneside_hd_fabric_values = result_values["fabric"]['one_side_hausdorff_cost_fabric_ls'].values
        oneside_hd_contact_values = result_values["contact"]['one_side_hausdorff_cost_contact_ls'].values
        # result_str += f' {fabric_type} {fabric_type} {fabric_type} {fabric_type}'
        # result_values_str += f' {chamfer_l1_fabric_values.mean()} {chamfer_l1_fabric_values.std()} {chamfer_l1_contact_values.mean()} {chamfer_l1_contact_values.std()}'
        # print(f"{key} & "
        # f"{np.round(chamfer_l1_fabric_values.mean(), decimals=nr_decimals)} \pm {np.round(chamfer_l1_contact_values.std(), decimals=nr_decimals)} & "
        # f"{np.round(oneside_hd_fabric_values.mean(), decimals=nr_decimals)} \pm {np.round(oneside_hd_contact_values.std(), decimals=nr_decimals)} & "
        #   )
        print(f"fabric order = {fabric_type}")
        result_str +=f"& {np.round(chamfer_l1_fabric_values.mean(), decimals=nr_decimals)} $\pm$ {np.round(chamfer_l1_contact_values.std(), decimals=nr_decimals)} "
        result_str +=f"& {np.round(chamfer_l1_contact_values.mean(), decimals=nr_decimals)} $\pm$ {np.round(chamfer_l1_contact_values.std(), decimals=nr_decimals)} "
        result_str +=f"& {np.round(oneside_hd_fabric_values.mean(), decimals=nr_decimals)} $\pm$ {np.round(oneside_hd_contact_values.std(), decimals=nr_decimals)} "
        result_str +=f"& {np.round(oneside_hd_contact_values.mean(), decimals=nr_decimals)} $\pm$ {np.round(oneside_hd_contact_values.std(), decimals=nr_decimals)} "
        
    # print(result_str)
        # print(f'\n sim={key}, key={fabric_type}')
        # print(f'Fabric = {result_values["fabric"].mean()}\n')
        # print(f'Contact = {result_values["contact"].mean()}\n')
        # print(f'Mean contact = {result_values["contact"].mean()} std={result_values["contact"].std()}')
        # print(f'Mean fabric = {result_values["fabric"].mean()} std={result_values["contact"].std()}')
        # print(f'Mean sum = {result_values["contact"].mean()+result_values["fabric"].mean()}')
        # if len(sim_results[env_name][fabric]) > 0:  # Concatenate it
        #     all_sim_results[env_name][fabric][result_type] = pd.concat((all_sim_results[env_name][fabric][result_type], result), axis=0)
        # else:  # Create a new one
        #     # print(f"New fabric = {fd}")
        #     sim_results[env_name][fabric][result_type] = result
        
print(result_str)
print(result_values_str)

In [11]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)
nr_decimals=3

In [ ]:
result_str = ''
result_values_str = ''

def get_result_string(results_dict, sim_name, fabric_type, primitive, metric_name, contact_type):
    # print(f"Evaluating= {sim_name} {fabric_type} {primitive} {metric_name}")
    result_values = results_dict[sim_name][fabric_type][primitive]
    error_values = result_values[metric_name].values
    value_str =f" {np.round(error_values.mean(), decimals=nr_decimals)} $\pm$ {np.round(error_values.std(), decimals=nr_decimals)} "
    
    return value_str

for fabric_type in ['cotton', 'chequered', 'linen']:
    # result_str += f'\n{fabric_type} '
    
    mujoco_cd_f = get_result_string(all_sim_results, 'mujoco', fabric_type, 'dynamic', 'chamfer_l1_cost_fabric_ls', 'fabric')
    mujoco_cd_c = get_result_string(all_sim_results, 'mujoco', fabric_type, 'quasi_static', 'chamfer_l1_cost_fabric_ls', 'contact')
    mujoco_hd_f = get_result_string(all_sim_results, 'mujoco', fabric_type, 'dynamic', 'one_side_hausdorff_cost_fabric_ls', 'fabric')
    mujoco_hd_c = get_result_string(all_sim_results, 'mujoco', fabric_type, 'quasi_static', 'one_side_hausdorff_cost_fabric_ls', 'contact')
    
    bullet_cd_f = get_result_string(all_sim_results, 'bullet', fabric_type, 'dynamic', 'chamfer_l1_cost_fabric_ls', 'fabric')
    bullet_cd_c = get_result_string(all_sim_results, 'bullet', fabric_type, 'quasi_static', 'chamfer_l1_cost_fabric_ls', 'contact')
    bullet_hd_f = get_result_string(all_sim_results, 'bullet', fabric_type, 'dynamic', 'one_side_hausdorff_cost_fabric_ls', 'fabric')
    bullet_hd_c = get_result_string(all_sim_results, 'bullet', fabric_type, 'quasi_static', 'one_side_hausdorff_cost_fabric_ls', 'contact')
        
    softgym_cd_f = get_result_string(all_sim_results, 'softgym', fabric_type, 'dynamic', 'chamfer_l1_cost_fabric_ls', 'fabric')
    softgym_cd_c = get_result_string(all_sim_results, 'softgym', fabric_type, 'quasi_static', 'chamfer_l1_cost_fabric_ls', 'contact')
    softgym_hd_f = get_result_string(all_sim_results, 'softgym', fabric_type, 'dynamic', 'one_side_hausdorff_cost_fabric_ls', 'fabric')
    softgym_hd_c = get_result_string(all_sim_results, 'softgym', fabric_type, 'quasi_static', 'one_side_hausdorff_cost_fabric_ls', 'contact')
        
    sofa_cd_f = get_result_string(all_sim_results, 'sofa', fabric_type, 'dynamic', 'chamfer_l1_cost_fabric_ls', 'fabric')
    sofa_cd_c = get_result_string(all_sim_results, 'sofa', fabric_type, 'quasi_static', 'chamfer_l1_cost_fabric_ls', 'contact')
    sofa_hd_f = get_result_string(all_sim_results, 'sofa', fabric_type, 'dynamic', 'one_side_hausdorff_cost_fabric_ls', 'fabric')
    sofa_hd_c = get_result_string(all_sim_results, 'sofa', fabric_type, 'quasi_static', 'one_side_hausdorff_cost_fabric_ls', 'contact')
    
    if fabric_type == 'linen':
        fabric_name = 'Linen'
    elif fabric_type == 'cotton':
        fabric_name = 'Towel'
    elif fabric_type == 'chequered':
        fabric_name = 'Cheq.'
    else:
        fabric_name = fabric_type
    
    # softgym_cd_c = None
    # softgym_hd_c = None
    
    # MuJoCo CD
    result_str +="\\multirow{4}{*}{\\textbf{" + f"{fabric_name}" + "}}"
    result_str +=" & $\\text{CD}_d$ & " f"{mujoco_cd_f} & {bullet_cd_f} & {softgym_cd_f} & {sofa_cd_f} \\\\ \n"
    result_str +=" & $\\text{HD}_d$ & " f"{mujoco_hd_f} & {bullet_hd_f} & {softgym_hd_f} & {sofa_hd_f} \\\\ \n"
    result_str += " \cmidrule{2-6} \n"
    result_str +=" & $\\text{CD}_q$ & " f"{mujoco_cd_c} & {bullet_cd_c} & {softgym_cd_c} & {sofa_cd_c} \\\\ \n"
    result_str +=" & $\\text{HD}_q$ & " f"{mujoco_hd_c} & {bullet_hd_c} & {softgym_hd_c} & {sofa_hd_c} \\\\ \n"
    result_str += " \midrule \n"

print(result_str)
    

In [ ]:
result_str = ''
for fabric_type in ['towel', 'chequered', 'white']:
    # result_str += f'\n{fabric_type} '
    
    mujoco_cd_f = get_result_string(all_sim_results, 'mujoco', fabric_type, 'chamfer_l1_cost_fabric_ls', 'fabric')
    mujoco_cd_c = get_result_string(all_sim_results, 'mujoco', fabric_type, 'chamfer_l1_cost_contact_ls', 'contact')
    mujoco_hd_f = get_result_string(all_sim_results, 'mujoco', fabric_type, 'chamfer_l2_cost_fabric_ls', 'fabric')
    mujoco_hd_c = get_result_string(all_sim_results, 'mujoco', fabric_type, 'chamfer_l2_cost_contact_ls', 'contact')
    
    bullet_cd_f = get_result_string(all_sim_results, 'bullet', fabric_type, 'chamfer_l1_cost_fabric_ls', 'fabric')
    bullet_cd_c = get_result_string(all_sim_results, 'bullet', fabric_type, 'chamfer_l1_cost_contact_ls', 'contact')
    bullet_hd_f = get_result_string(all_sim_results, 'bullet', fabric_type, 'chamfer_l2_cost_fabric_ls', 'fabric')
    bullet_hd_c = get_result_string(all_sim_results, 'bullet', fabric_type, 'chamfer_l2_cost_contact_ls', 'contact')
        
    softgym_cd_f = get_result_string(all_sim_results, 'softgym', fabric_type, 'chamfer_l1_cost_fabric_ls', 'fabric')
    softgym_cd_c = get_result_string(all_sim_results, 'softgym', fabric_type, 'chamfer_l1_cost_contact_ls', 'contact')
    softgym_hd_f = get_result_string(all_sim_results, 'softgym', fabric_type, 'chamfer_l2_cost_fabric_ls', 'fabric')
    softgym_hd_c = get_result_string(all_sim_results, 'softgym', fabric_type, 'chamfer_l2_cost_contact_ls', 'contact')
        
    sofa_cd_f = get_result_string(all_sim_results, 'sofa', fabric_type, 'chamfer_l1_cost_fabric_ls', 'fabric')
    sofa_cd_c = get_result_string(all_sim_results, 'sofa', fabric_type, 'chamfer_l1_cost_contact_ls', 'contact')
    sofa_hd_f = get_result_string(all_sim_results, 'sofa', fabric_type, 'chamfer_l2_cost_fabric_ls', 'fabric')
    sofa_hd_c = get_result_string(all_sim_results, 'sofa', fabric_type, 'chamfer_l2_cost_contact_ls', 'contact')
    
    if fabric_type == 'white':
        fabric_name = 'linen'
    else:
        fabric_name = fabric_type
    
    # MuJoCo CD
    result_str +=f"{fabric_name}" "& $\\text{CD}_f$ & " f"{mujoco_cd_f} & {bullet_cd_f} & {softgym_cd_f} & {sofa_cd_f} \\\\ \n"
    result_str +=f"{fabric_name}" "& $\\text{CD}_c$ & " f"{mujoco_cd_c} & {bullet_cd_c} & {softgym_cd_c} & {sofa_cd_c} \\\\ \n"
    result_str +=f"{fabric_name}" "& $\\text{HD}_f$ & " f"{mujoco_hd_f} & {bullet_hd_f} & {softgym_hd_f} & {sofa_hd_f} \\\\ \n"
    result_str +=f"{fabric_name}" "& $\\text{HD}_c$ & " f"{mujoco_hd_c} & {bullet_hd_c} & {softgym_hd_c} & {sofa_hd_c} \\\\ \n"

print(result_str)

# Measure the difference between the different clouds

In [4]:
from bcm.vision.utils import chamfer_distance_w_unidirectional, pcu_unidirectional_chamfer_distance
from bcm.utils import load_targets
import torch
import point_cloud_utils as pcu


In [5]:
data_path = "/path_to_your_projects/datasets/Dynamic_cloth_bench/demos"
chequered_names = ["chequered_rag_0"]
towel_names = ["towel_rag_0"]
white_names = ["white_rag_0"]



In [6]:
# Measure the difference between chequered and towel. Target is Chequered, so Towel -> Chequered
def get_source_target_distance(source_names, target_names):
    dist_source_to_target = {
        'fabric_cd':[],
        'contact_cd': [],
        'fabric_hd':[],
        'contact_hd': []
    }
    
    for target in target_names:
        
        target_path = data_path + "/" + target + "/cloud/"
        target_tgt = load_targets(target_path)
        
        target_len = len(target_tgt)
        
        for source in source_names:
            source_path = data_path + "/" + source + "/cloud/"
            source_tgt = load_targets(source_path)
            
            source_len = len(source_tgt)
            
            if target_len > source_len:
                total_length = source_len
            else:
                total_length = target_len
            
            # Compare all the points
            for i in range(total_length):
                # print(f"Comparing timestep={i}")
                source_cloud = torch.from_numpy(source_tgt[i]).unsqueeze(0).float()
                target_cloud = torch.from_numpy(target_tgt[i]).unsqueeze(0).float()
                x_cloud = tranform_target_to_mujoco(source_cloud)
                y_cloud = tranform_target_to_mujoco(target_cloud)
                
                cd_cost_l1 = chamfer_distance_w_unidirectional(x_cloud, y_cloud, single_directional=True, norm=1)[0]
                hd_p1_to_p2 = pcu.one_sided_hausdorff_distance(x_cloud.numpy()[0], y_cloud.numpy()[0], return_index=False)
            
                if i> 59:
                    dist_source_to_target['contact_cd'].append(cd_cost_l1)
                    dist_source_to_target['contact_hd'].append(hd_p1_to_p2)
                else:
                    dist_source_to_target['fabric_cd'].append(cd_cost_l1)
                    dist_source_to_target['fabric_hd'].append(hd_p1_to_p2)
    
    return dist_source_to_target


In [48]:
all_chequered_names = ["chequered_rag_0", "chequered_rag_1", "chequered_rag_2"]
all_towel_names = ["towel_rag_0", "towel_rag_1", "towel_rag_2"]
all_white_names = ["white_rag_0", "white_rag_1", "white_rag_2"]

# Plot both sim and target

In [7]:
import open3d as o3d


In [ ]:
data_path = "/path_to_your_projects/datasets/Dynamic_cloth_bench/demos"

target = "cotton_rag_0"

source_dir = "/path_to_your_projects/projects/dynamic_dual_manip/results_npy_27/"

primitive = 'quasi_static'
primitive = 'dynamic'

target_path = data_path + "/" + target + "/cloud/"
target_tgt = load_targets(target_path)


# Definition for Bullet
# pos_z_table = 0.195
# pos_table_sofa = 0.2
# 
# table_x_i = np.linspace(start=-0.4, stop=0.4, num=20)
# table_y_i = np.linspace(start=-0.1, stop=0.8, num=20)
# tab_x, tab_y = np.meshgrid(table_x_i, table_y_i, indexing='ij')
# tab_xy = np.vstack((tab_x.flatten(), tab_y.flatten()))
# tab_z = np.zeros((1, tab_xy.shape[1])) + pos_z_table
# 
# # Define the table
# table_points = np.vstack((tab_xy, tab_z)).T
# table_points_sofa = np.vstack((tab_xy, np.zeros((1, tab_xy.shape[1])) + pos_table_sofa)).T

print(f"Target length = {len(target_tgt)}")

In [9]:
def get_target_and_sim_cloud(sim_cloud_traj, timestep):
    
    # print(f"Sim = {sim} shape source = {sim_cloud_traj.shape[0]}")
    
    # Define the source to target match 
    # target_it = np.linspace(0, sim_cloud_traj.shape[0], len(target_tgt), dtype=int)
    
    skip_frames = 0
    
    # Load the target
    freq_relation = sim_cloud_traj.shape[0]/len(target_tgt)
    
    iteration_nr = int(timestep  * freq_relation)
    
    # target_idx = int(target_it[iteration_nr] - skip_frames)
    target_idx = int(iteration_nr/freq_relation - skip_frames)
    # print(f"Freq relation={freq_relation}, iteration = {iteration_nr} target idx={target_idx}")

    target_cloud = torch.from_numpy(target_tgt[target_idx]).unsqueeze(0).float()
    
    target_vertices = target_cloud.numpy()[0]
    
    # Select the source cloud to match
    # source_nr = target_it[iteration_nr]
    sim_cloud_npy = sim_cloud_traj[iteration_nr]
    sim_cloud_torch = torch.from_numpy(sim_cloud_npy).unsqueeze(0)
    
    # print(f"Shape sim = {sim_cloud_torch.shape} taret={transformed_target.shape}")
    cd_cost_l1 = chamfer_distance_w_unidirectional(sim_cloud_torch.type(torch.FloatTensor), target_cloud.type(torch.FloatTensor), single_directional=True, norm=1)[0]
    if sim_cloud_npy.shape[0] < target_vertices.shape[0]:
        first_sim_cloud = target_vertices.copy()
        first_sim_cloud[:sim_cloud_npy.shape[0], :] = sim_cloud_npy
        first_sim_cloud[sim_cloud_npy.shape[0]:, :] = np.nan
        sim_cloud_npy = first_sim_cloud[~np.isnan(first_sim_cloud[:, 0]), :]
    else:
        first_sim_cloud = sim_cloud_npy.copy()
        first_sim_cloud[:target_vertices.shape[0], :] = target_vertices
        first_sim_cloud[target_vertices.shape[0]:, :] = np.nan
        target_vertices = first_sim_cloud[~np.isnan(first_sim_cloud[:, 0]), :]
    hd_p1_to_p2 = pcu.one_sided_hausdorff_distance(sim_cloud_npy, target_vertices, return_index=False)
    # print(f"Sim = {sim}, nr points = {sim_cloud_npy.shape[0]}, target points={target_vertices.shape[0]}")
    # print(f"Sim = {sim}, cd_cost_l1 = {cd_cost_l1}, hd_p1_to_p2={hd_p1_to_p2}, iteration = {iteration_nr} target idx={target_idx}")
    
    # Reduce the number of points so that they can be plotted
    source_temp = o3d.geometry.PointCloud()
    source_temp.points = o3d.utility.Vector3dVector(sim_cloud_npy)
    downpcd = source_temp.voxel_down_sample(voxel_size=0.02)
    
    min_voxel_size = 0.025
    voxel_step = 0.005
    target_shape = 500
    
    if np.asarray(downpcd.points).shape[0] > target_shape:
        downpcd = downpcd.voxel_down_sample(voxel_size=min_voxel_size+voxel_step)
    
    sim_cloud_reduced = np.asarray(downpcd.points)
    
    target_temp = o3d.geometry.PointCloud()
    target_temp.points = o3d.utility.Vector3dVector(target_vertices)
    # 0.03 500 - 1k, 1k4 
    # 0.02 = 1-2k points, 2k
    # 0.01 = ~5-6k
    
    target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size)
    if np.asarray(target_downpcd.points).shape[0] > target_shape:
        target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step)
        if np.asarray(target_downpcd.points).shape[0] > target_shape:
            target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*2)
        if np.asarray(target_downpcd.points).shape[0] > target_shape:
                target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*3)
                if np.asarray(target_downpcd.points).shape[0] > target_shape:
                    target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*4)
        
    
    target_cloud_reduced = np.asarray(target_downpcd.points)
    
    # print(f"Reduced cloud sim nr points = {sim_cloud_reduced.shape[0]}, target points={target_cloud_reduced.shape[0]}")
    
    
    # return target_cloud_reduced, sim_cloud_reduced, cd_cost_l1, hd_p1_to_p2
    return target_cloud_reduced, sim_cloud_npy, cd_cost_l1, hd_p1_to_p2


In [ ]:
%matplotlib inline
# Real, t=0 timestep=11, t=4.4 timestep=131

real_time = np.around(np.linspace(start=0.0, stop=4.4, num=121), decimals=2)

# times_to_plot = [0.7, 1.0, 1.4, 2.0, 2.5, 3.0, 3.5, 4.0]
# times_to_plot = [0.0, 0.7, 1.0, 1.4, 2.0, 2.5, 3.0, 3.5, 4.4]
times_to_plot = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.4]
times_to_plot_quasi = []
times_to_plot_dyn = []

width=10*len(times_to_plot)
height = 10
# sim='mujoco'
fig, ax = plt.subplots(4, len(times_to_plot), figsize=(width,height), subplot_kw=dict(projection='3d'), dpi=300)
i = 0
elev = 8
azim = 13
for sim in sim_folders:
    j=0
    for time in times_to_plot:
        if time < 2.2:
            alpha_red = 0.2
            alpha_black = 1.0
        else:
            alpha_red = 1.0
            alpha_black = 0.1
        idx = (np.abs(real_time - time)).argmin()
        # print(f'idx={idx}, timevalue={real_time[idx]}')
        skip_frames = 0
        target_idx = idx + skip_frames
        
        print(f"Sim={sim} time={time}")
        source_path = source_dir + sim + '_towel_rag_0_1050_cloth_sequence.npy'
    
        sim_cloud_traj = np.load(source_path)
        target_vertices, sim_cloud_npy, _, _ = get_target_and_sim_cloud(sim_cloud_traj, target_idx)
        
        ax[i, j].scatter(sim_cloud_npy[:, 0], sim_cloud_npy[:, 1], sim_cloud_npy[:, 2],
                   marker='*', color='r', label='Sim', alpha=alpha_red)
        
        ax[i, j].scatter(target_vertices[:, 0], target_vertices[:, 1], target_vertices[:, 2],
                   marker='^', color='k', label='Real', alpha=alpha_black)
        
        # ax[i].set_title(sim)
        ax[i, j].grid(visible=False)
        ax[i, j].view_init(elev=elev, azim=azim)
        ax[i, j].set_xlim([-0.4, 0.4])
        ax[i, j].set_ylim([-0.8, 0.8])
        ax[i, j].set_zlim([0.00, 1.1])
        ax[i, j].set_yticklabels([])
        ax[i, j].set_xticklabels([])
        ax[i, j].set_aspect('equal')
        # ax[i, j].set_xticks([])
        # ax[i, j].set_yticks([])
        # plt.gca().axes.get_yaxis().set_visible(False)
        # ax[i,j ].set_axis_off()
        
        j+=1
    
    i+=1
    # plt.suptitle(f'Sim={sim}')
# plt.subplots_adjust(wspace=0, hspace=0)

# plt.savefig(f'comparison_clouds.pdf')
plt.show()

# Evaluate the time taken per simulation and frequency  

In [2]:
target = "towel_rag_0"

source_dir_1KHz = "/path_to_your_projects/projects/dynamic_dual_manip/results_1KHz/"
source_dir_100Hz = "/path_to_your_projects/projects/dynamic_dual_manip/results_npy_27/"
source_dir_10Hz = "/path_to_your_projects/projects/dynamic_dual_manip/results_10Hz/"

# results_path = "/path_to_your_projects/projects/dynamic_dual_manip/RESULTS/13_sept_results/"

def get_moving_filter(cloth_sequence):
    
    diff_list = []
    for i in range(1, cloth_sequence.shape[0]-1):
        averaged_filter_pose = (cloth_sequence[i-1] + cloth_sequence[i] + cloth_sequence[i+1])/3
        diff_timepoint = np.abs(averaged_filter_pose - cloth_sequence[i])
        diff_timepoint_mean = diff_timepoint.mean()
        diff_list.append(diff_timepoint_mean)
    
    return np.array(diff_list)


def get_time_dict(source_path):
    results = {
        'time':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'filter':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'cd_d_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'cd_q_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'hd_f_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
        'hd_c_loss':  {
            'mujoco': [],
            'bullet': [],
            'softgym': [],
            'sofa': [],
        },
    }

    
    for fd in os.listdir(source_path):
        if 'time' in fd or 'sequence' in fd:
            full_path = source_path+fd
            # if 'time' in fd:
            #     continue
            # print(f"full path={full_path}")
            if 'time' in fd:
                key_dict = 'time'
                result_arr = np.load(full_path)
            else:
                key_dict = 'filter'
                cloth_sequence = np.load(full_path)
                result_arr = get_moving_filter(cloth_sequence)
                if 'quasi_static' in fd:
                    continue  # We compute the moving filter only for the dynamic
            
            if  'mujoco' in fd:
                results[key_dict]['mujoco'].append(result_arr)
            elif  'bullet' in fd:
                results[key_dict]['bullet'].append(result_arr)
            elif 'softgym' in fd:
                results[key_dict]['softgym'].append(result_arr)
            else:
                results[key_dict]['sofa'].append(result_arr)
    
    for key, val in results['time'].items():
        results['time'][key] = np.hstack(val)
    for key, val in results['filter'].items():
        results['filter'][key] = np.hstack(val)
    
    frequency_metric_results = { 'mujoco': {}, 'bullet': {}, 'softgym': {}, 'sofa': {}}
    
    # for result in sim_folders:
    for fd in os.listdir(source_path):
        if 'csv' in fd:
            pass
        else:
            continue
            
        if 'mujoco' in fd:
            env_name = 'mujoco'
        elif 'bullet' in fd:
            env_name = 'bullet'
        elif 'softgym' in fd:
            env_name = 'softgym'
        else:
            env_name = 'sofa'
        
        if 'dynamic' in fd:
            primitive = 'dynamic'
        else:
            primitive = 'quasi_static'

        
        # result = np.load(f'{results_path}/{fd}')
        result = pd.read_csv(f'{source_path}/{fd}')
        if primitive in  frequency_metric_results[env_name]:  # Concatenate it
            frequency_metric_results[env_name][primitive] = pd.concat((frequency_metric_results[env_name][primitive], result), axis=0)
        else:  # Create a new one
            frequency_metric_results[env_name][primitive] = result
    for key, val in frequency_metric_results.items():
        results['cd_d_loss'][key] = val["dynamic"]['chamfer_l1_cost_fabric_ls'].values
        results['cd_q_loss'][key] = val["quasi_static"]['chamfer_l1_cost_contact_ls'].values
        # results['hd_f_loss'][key] = val["fabric"]['one_side_hausdorff_cost_fabric_ls'].values
        # results['hd_c_loss'][key] = val["contact"]['one_side_hausdorff_cost_contact_ls'].values
    
    
    return results


In [ ]:
time_dict_10Hz = get_time_dict(source_dir_10Hz)
# time_dict_100Hz = get_time_dict(source_dir_100Hz)
# time_dict_1KHz = get_time_dict(source_dir_1KHz)
                

In [ ]:
for key, _  in time_dict_1KHz.items():
    print(f"Sim keys={key}")

print("1KHz results")

result_10Hz_str = "\multirow{2}{*}{10} & Step Time (ms.)"
for key, val in time_dict_10Hz['time'].items():
    val_ms = val * 1000
    mean_val = np.round(val_ms.mean(), decimals=1)
    std_val = np.round(val_ms.std(), decimals=1)
    result_10Hz_str += f" & {mean_val} $\pm$ {std_val}"
result_10Hz_str += "\\\\ \n  & $\L_\\text{s}$ (10e3)"
for key, val in time_dict_10Hz['filter'].items():
    val_e6 = val * 1e6
    mean_val = np.round(val_e6.mean(), decimals=2)
    std_val = np.round(val_e6.std(), decimals=2)
    result_10Hz_str += f" & {mean_val} $\pm$ {std_val}"

result_10Hz_str += " \\\\"

result_100Hz_str = "\multirow{2}{*}{100} & Step Time (ms.)"
for key, val in time_dict_100Hz['time'].items():
    val_ms = val * 1000
    mean_val = np.round(val_ms.mean(), decimals=1)
    std_val = np.round(val_ms.std(), decimals=1)
    result_100Hz_str += f" & {mean_val} $\pm$ {std_val}"
result_100Hz_str += "\\\\ \n  & $\L_\\text{s}$ (10e6)"
for key, val in time_dict_100Hz['filter'].items():
    val_e6 = val * 1e6
    mean_val = np.round(val_e6.mean(), decimals=2)
    std_val = np.round(val_e6.std(), decimals=2)
    result_100Hz_str += f" & {mean_val} $\pm$ {std_val}"

result_100Hz_str += " \\\\"


result_1KHz_str = "\multirow{2}{*}{1000} & Step Time (ms.)"
for key, val in time_dict_1KHz['time'].items():
    val_ms = val * 1000
    mean_val = np.round(val_ms.mean(), decimals=1)
    std_val = np.round(val_ms.std(), decimals=1)
    result_1KHz_str += f" & {mean_val} $\pm$ {std_val}"
result_1KHz_str+= "\\\\ \n  & $\L_\\text{s}$ (10e6)"
for key, val in time_dict_1KHz['filter'].items():
    val_e6 = val * 1e6
    mean_val = np.round(val_e6.mean(), decimals=2)
    std_val = np.round(val_e6.std(), decimals=2)
    result_1KHz_str += f" & {mean_val} $\pm$ {std_val}"

result_1KHz_str += " \\\\"

print(result_10Hz_str)
print("\\midrule")
print(result_100Hz_str)
print("\\midrule")
print(result_1KHz_str)
print("\\bottomrule")

In [47]:
# Mean time - X=10 Hz, 100 Hz, 1KHz. Y value = mujoco, bullet, flex, sofa 
mean_time = np.array([
    [np.array(time_dict_10Hz['time']['mujoco']).mean(),
     np.array(time_dict_100Hz['time']['mujoco']).mean(),
     np.array(time_dict_1KHz['time']['mujoco']).mean()],
    [np.array(time_dict_10Hz['time']['bullet']).mean(),
     np.array(time_dict_100Hz['time']['bullet']).mean(),
     np.array(time_dict_1KHz['time']['bullet']).mean()],
    [np.array(time_dict_10Hz['time']['softgym']).mean(),
     np.array(time_dict_100Hz['time']['softgym']).mean(),
     np.array(time_dict_1KHz['time']['softgym']).mean()],
    [np.array(time_dict_10Hz['time']['sofa']).mean(),
     np.array(time_dict_100Hz['time']['sofa']).mean(),
     np.array(time_dict_1KHz['time']['sofa']).mean()]
])

mean_filter = np.array([
    [np.array(time_dict_10Hz['filter']['mujoco']).mean(),
     np.array(time_dict_100Hz['filter']['mujoco']).mean(),
     np.array(time_dict_1KHz['filter']['mujoco']).mean()],
    [np.array(time_dict_10Hz['filter']['bullet']).mean(),
     np.array(time_dict_100Hz['filter']['bullet']).mean(),
     np.array(time_dict_1KHz['filter']['bullet']).mean()],
    [np.array(time_dict_10Hz['filter']['softgym']).mean(),
     np.array(time_dict_100Hz['filter']['softgym']).mean(),
     np.array(time_dict_1KHz['filter']['softgym']).mean()],
    [np.array(time_dict_10Hz['filter']['sofa']).mean(),
     np.array(time_dict_100Hz['filter']['sofa']).mean(),
     np.array(time_dict_1KHz['filter']['sofa']).mean()]
])

mean_cd = np.array([
    [np.array(time_dict_10Hz['cd_f_loss']['mujoco']).mean() + np.array(time_dict_10Hz['cd_c_loss']['mujoco']).mean(),
     np.array( time_dict_100Hz['cd_f_loss']['mujoco']).mean() + np.array(time_dict_100Hz['cd_c_loss']['mujoco']).mean(),
     np.array( time_dict_1KHz['cd_f_loss']['mujoco']).mean() + np.array(time_dict_1KHz['cd_c_loss']['mujoco']).mean()],
   [np.array( time_dict_10Hz['cd_f_loss']['bullet']).mean() + np.array(time_dict_10Hz['cd_c_loss']['bullet']).mean(),
     np.array( time_dict_100Hz['cd_f_loss']['bullet']).mean() + np.array(time_dict_100Hz['cd_c_loss']['bullet']).mean(),
     np.array( time_dict_1KHz['cd_f_loss']['bullet']).mean() + np.array(time_dict_1KHz['cd_c_loss']['bullet']).mean()],
    [np.array( time_dict_10Hz['cd_f_loss']['softgym']).mean() + np.array(time_dict_10Hz['cd_c_loss']['softgym']).mean(),
     np.array( time_dict_100Hz['cd_f_loss']['softgym']).mean() + np.array(time_dict_100Hz['cd_c_loss']['softgym']).mean(),
     np.array( time_dict_1KHz['cd_f_loss']['softgym']).mean() + np.array(time_dict_1KHz['cd_c_loss']['softgym']).mean()],
    [np.array( time_dict_10Hz['cd_f_loss']['sofa']).mean() + np.array(time_dict_10Hz['cd_c_loss']['sofa']).mean(),
     np.array( time_dict_100Hz['cd_f_loss']['sofa']).mean() + np.array(time_dict_100Hz['cd_c_loss']['sofa']).mean(),
     np.array( time_dict_1KHz['cd_f_loss']['sofa']).mean() + np.array(time_dict_1KHz['cd_c_loss']['sofa']).mean()],
])

mean_hd = np.array([
    [np.array( time_dict_10Hz['hd_f_loss']['mujoco']).mean() + np.array(time_dict_10Hz['hd_c_loss']['mujoco']).mean(),
     np.array( time_dict_100Hz['hd_f_loss']['mujoco']).mean() + np.array(time_dict_100Hz['hd_c_loss']['mujoco']).mean(),
     np.array( time_dict_1KHz['hd_f_loss']['mujoco']).mean() + np.array(time_dict_1KHz['hd_c_loss']['mujoco']).mean()],
   [np.array( time_dict_10Hz['hd_f_loss']['bullet']).mean() + np.array(time_dict_10Hz['hd_c_loss']['bullet']).mean(),
     np.array( time_dict_100Hz['hd_f_loss']['bullet']).mean() + np.array(time_dict_100Hz['hd_c_loss']['bullet']).mean(),
     np.array( time_dict_1KHz['hd_f_loss']['bullet']).mean() + np.array(time_dict_1KHz['hd_c_loss']['bullet']).mean()],
    [np.array( time_dict_10Hz['hd_f_loss']['softgym']).mean() + np.array(time_dict_10Hz['hd_c_loss']['softgym']).mean(),
     np.array( time_dict_100Hz['hd_f_loss']['softgym']).mean() + np.array(time_dict_100Hz['hd_c_loss']['softgym']).mean(),
     np.array( time_dict_1KHz['hd_f_loss']['softgym']).mean() + np.array(time_dict_1KHz['hd_c_loss']['softgym']).mean()],
    [np.array( time_dict_10Hz['hd_f_loss']['sofa']).mean() + np.array(time_dict_10Hz['hd_c_loss']['sofa']).mean(),
     np.array( time_dict_100Hz['hd_f_loss']['sofa']).mean() + np.array(time_dict_100Hz['hd_c_loss']['sofa']).mean(),
     np.array( time_dict_1KHz['hd_f_loss']['sofa']).mean() + np.array(time_dict_1KHz['hd_c_loss']['sofa']).mean()],
])



In [68]:
%matplotlib inline
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

from matplotlib import pyplot as plt
# from matplotlib import rcParams
matplotlib.rcParams.update(matplotlib.rcParamsDefault)


def plot_mean_log_figure(name, results_array, log=True, second_scale=False):
    fig1, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150)
    
    x_time = np.arange(3)
    y_labels = sim_folders
    # Mujoco = blue, Bullet= gray, softgym=green, sofa=orange
    colours = ['#4986ae', '#c4c9d8', '#78b26d', '#f37738']
    for i in range(results_array.shape[0]):
        ax.plot(x_time, results_array[i, :], label=y_labels[i], marker='', linewidth=5, color=colours[i])
    # ax.plot(x_time, mean_time[0, :], label=y_labels[0], color='red', marker='', linewidth=5)
    # print(f"mujoco={mean_time[0, :]}")
    # ax.set_xticklabels(['0', '10', '1000'])
    # ax.set_yticklabels([])
    # plt.legend()
    ax.tick_params(axis='both', width=3, length=15, which='major')
    ax.tick_params(axis='both', width=1, length=7.5, which='minor')
    if log:
        plt.yscale('log')#, linthresh=0.01)
        if second_scale:
            ax.set_ylim(1e-3, 10.0)
        else:
            ax.set_ylim(1e-7, 10.0)
    else: 
        if  second_scale:
            ax.set_ylim(0.15, 0.35)
        else:
            ax.set_ylim(0.5, 0.8)

    plt.subplots_adjust(left=0.14, right=0.95, bottom=0.1, top=0.95)
    # plt.rc('axes', labelsize=SMALL_SIZE)          # controls default text sizes
    plt.savefig(f'freq_results_{name}.pdf', dpi=300)
    # plt.show()


# plot_mean_log_figure(name='time', results_array=mean_time, second_scale=True)
plot_mean_log_figure(name='filter', results_array=mean_filter)
# plot_mean_log_figure(name='cd', results_array=mean_cd, log=False, second_scale=True)
# plot_mean_log_figure(name='hd', results_array=mean_hd, log=False)


# Plot both sim and target

In [3]:
import open3d as o3d
import torch
import point_cloud_utils as pcu
from bcm.vision.utils import chamfer_distance_w_unidirectional, pcu_unidirectional_chamfer_distance
from bcm.utils import load_targets, get_dynamic_target_cutoff


In [36]:
def get_target_and_sim_cloud(sim_cloud_traj, timestep, skip_frames, primitive):
    
    
    # Load the target
    if primitive == 'quasi_static':
        freq_relation = sim_cloud_traj.shape[0]/420
    else:
        freq_relation = sim_cloud_traj.shape[0]/len(target_tgt)
    
    # target_comparison = (60 + skip_frames) * freq_relation
    
    iteration_nr = int(timestep  * freq_relation)
    
    # target_idx = int(target_it[iteration_nr] - skip_frames)
    target_idx = int(iteration_nr/freq_relation - skip_frames)
    # print(f"Target idx={target_idx}, target compare = {target_comparison}")
    if target_idx < 110:
        # print(f"Free dynamics")
        fabric_w_contact = False
    else:
        # print(f"Contact")
        fabric_w_contact = True
    # print(f"Freq relation={freq_relation}, iteration = {iteration_nr} target idx={target_idx}")

    target_cloud = torch.from_numpy(target_tgt[target_idx]).unsqueeze(0).float()
    
    # transformed_target = tranform_target_to_mujoco(target_cloud)
    target_vertices = target_cloud.numpy()[0]
    
    # Select the source cloud to match
    # source_nr = target_it[iteration_nr]
    sim_cloud_npy = sim_cloud_traj[iteration_nr]
    sim_cloud_torch = torch.from_numpy(sim_cloud_npy).unsqueeze(0)
    
    # print(f"Shape sim = {sim_cloud_torch.shape} taret={transformed_target.shape}")
    cd_cost_l1 = chamfer_distance_w_unidirectional(sim_cloud_torch.type(torch.FloatTensor), target_cloud.type(torch.FloatTensor), single_directional=True, norm=1)[0]
    if sim_cloud_npy.shape[0] < target_vertices.shape[0]:
        first_sim_cloud = target_vertices.copy()
        first_sim_cloud[:sim_cloud_npy.shape[0], :] = sim_cloud_npy
        first_sim_cloud[sim_cloud_npy.shape[0]:, :] = np.nan
        sim_cloud_npy = first_sim_cloud[~np.isnan(first_sim_cloud[:, 0]), :]
    else:
        first_sim_cloud = sim_cloud_npy.copy()
        first_sim_cloud[:target_vertices.shape[0], :] = target_vertices
        first_sim_cloud[target_vertices.shape[0]:, :] = np.nan
        target_vertices = first_sim_cloud[~np.isnan(first_sim_cloud[:, 0]), :]
    hd_p1_to_p2 = pcu.one_sided_hausdorff_distance(sim_cloud_npy, target_vertices, return_index=False)
    # print(f"Sim = {sim}, nr points = {sim_cloud_npy.shape[0]}, target points={target_vertices.shape[0]}")
    # print(f"Sim = {sim}, cd_cost_l1 = {cd_cost_l1}, hd_p1_to_p2={hd_p1_to_p2}, iteration = {iteration_nr} target idx={target_idx}")
    
    # Reduce the number of points so that they can be plotted
    source_temp = o3d.geometry.PointCloud()
    source_temp.points = o3d.utility.Vector3dVector(sim_cloud_npy)
    downpcd = source_temp.voxel_down_sample(voxel_size=0.02)
    
    min_voxel_size = 0.025
    voxel_step = 0.005
    target_shape = 1000
    
    if np.asarray(downpcd.points).shape[0] > target_shape:
        downpcd = downpcd.voxel_down_sample(voxel_size=min_voxel_size+voxel_step)
    
    sim_cloud_reduced = np.asarray(downpcd.points)
    
    target_temp = o3d.geometry.PointCloud()
    target_temp.points = o3d.utility.Vector3dVector(target_vertices)
    # 0.03 500 - 1k, 1k4 
    # 0.02 = 1-2k points, 2k
    # 0.01 = ~5-6k
    
    # print(f"Target size={target_vertices.shape}")
    
    target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size)
    if np.asarray(target_downpcd.points).shape[0] > target_shape:
        target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step)
        if np.asarray(target_downpcd.points).shape[0] > target_shape:
            target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*2)
        if np.asarray(target_downpcd.points).shape[0] > target_shape:
                target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*3)
                if np.asarray(target_downpcd.points).shape[0] > target_shape:
                    target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*4)
        
    
    target_cloud_reduced = np.asarray(target_downpcd.points)
    # print(f"Target reduced size={target_cloud_reduced.shape}")
    
    # print(f"Reduced cloud sim nr points = {sim_cloud_reduced.shape[0]}, target points={target_cloud_reduced.shape[0]}")
    
    
    # return target_cloud_reduced, sim_cloud_reduced, cd_cost_l1, hd_p1_to_p2
    return target_cloud_reduced, sim_cloud_npy, cd_cost_l1, hd_p1_to_p2, fabric_w_contact
    #return target_cloud_reduced, sim_cloud_npy, None, None, fabric_w_contact


In [5]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)
nr_decimals=3

In [53]:
def get_alphas(sim_name):
    if sim_name == 'mujoco3':
        # alpha_red = 0.5  # Looks good
        alpha_red = 0.08
        alpha_black = 0.05
    elif sim_name == 'bullet':
        alpha_red = 0.08  # Maybe a bit less?
        # alpha_red = 0.15  # looks ok-ish Maybe a bit less?
        alpha_black = 0.05
    elif sim_name == 'softgym':
        # alpha_red = 0.01 # not too bad, a bit too much
        # alpha_red = 0.05 # too much
        # print(f"Sotgym alphas")
        alpha_red = 0.01
        alpha_black = 0.05
    else: # sofa
        alpha_red = 0.08
        alpha_black = 0.05
        
    return alpha_red, alpha_black

def plot_and_save_cloth_sequence(cloth_sequence, sim_name, out_folder, chequered_type:str,
                                 primitive:str, elev=0, azim=90, plot=True):
    print(f"Evaluating primitive type={primitive}")
    if primitive == 'dynamic':
        real_time = np.around(np.linspace(start=0.0, stop=4.4, num=121), decimals=2)
    else:
        real_time = np.around(np.linspace(start=0.0, stop=14, num=420), decimals=2)
    skip_frames = 0

    
    switching_timestep = get_dynamic_target_cutoff(target_name=chequered_type,
                                                   primitive=primitive)

    cd_f_list = []
    hd_f_list = []
    alpha_red, alpha_black = get_alphas(sim_name)
    # elev = 0
    # elev = 8
    # azim = 0
    # azim = 13
    # elev = 0
    # azim = 90
    
    # skip_frames = 11
    # it_times = np.arange(30)
    # it_times_all = np.hstack((it_times, it_times+60))
    
    for i in range(real_time.shape[0]):
    # for i in range(real_time.shape[0]-10, real_time.shape[0]):
    # for i in range(20):
    # for i in it_times_all:
    # for i in range(2):
        if i == switching_timestep:
            break
        
        time = real_time[i]
        # if time < 2.2:
        
        idx = (np.abs(real_time - time)).argmin()
        # print(f'idx={idx}, timevalue={real_time[idx]}')
        
        target_idx = idx + skip_frames
        # print(f"Sim={sim_name} time={time}")
        
        (target_vertices, sim_cloud_npy, 
         cd_cost_l1, hd_p1_to_p2, fabric_w_contact) = get_target_and_sim_cloud(cloth_sequence, target_idx,
                                                                               skip_frames, primitive)
        
        if plot:
            fig, ax = plt.subplots(1, 1, figsize=(5, 5), subplot_kw=dict(projection='3d'), dpi=150)
            ax.scatter(sim_cloud_npy[:, 0], sim_cloud_npy[:, 1], sim_cloud_npy[:, 2],
                       marker='*', color='r', label='Sim', alpha=alpha_red)
            ax.scatter(target_vertices[:, 0], target_vertices[:, 1], target_vertices[:, 2],
                       marker='^', color='k', label='Real', alpha=alpha_black)
            
            ax.grid(visible=False)
            ax.view_init(elev=elev, azim=azim)
            if primitive == 'quasi_static':
                ax.set_xlim([-0.1, 0.6])
                ax.set_ylim([-0.7, 0.1])
                ax.set_zlim([0.00, 1.1])
            else:
                ax.set_xlim([-0.4, 0.4])
                ax.set_ylim([-0.8, 0.8])
                ax.set_zlim([0.00, 1.1])
            ax.set_xticklabels([])
            # ax.set_xlabel('X (m)')
            ax.set_ylabel('Y (m.)')
            ax.set_zlabel('Z (m.)')
            ax.set_aspect('auto')
            # Place the legend at the bottom
            # ax.legend(loc='upper center', bbox_to_anchor=(0.5, 0.01), fancybox=True, shadow=True, ncol=2)
            # Place the legend at the top
            # TODO MAY WANT TO ADD BACK THE LEGEND
            # ax.legend(loc='upper center', bbox_to_anchor=(0.5, 1.0), fancybox=True, shadow=True, ncol=2,
            #           fontsize=10)
            
            # ax.tick_params(axis='both', direction='out', reset=True, width=20, length=100, which='major')
            # ax.tick_params(axis='y', width=20, length=25, which='major')
            # ax.tick_params(axis='z', width=20, length=25, which='major')
            # ax.tick_params(axis='z', width=5, length=7.5, which='minor')
            
            
            # Write CD
            if primitive == 'quasi_static':
                plt.subplots_adjust(left=0.08, right=0.98, bottom=0.1, top=0.99)
                ax.text(0.5, -0.3, -0.45, s=f'CD={trunc(np.round(cd_cost_l1.item(), decimals=nr_decimals), decs=nr_decimals)}', fontsize=12)
                ax.text(0.5, -0.3, -0.55, s=f'HD={trunc(np.round(hd_p1_to_p2, decimals=nr_decimals), decs=nr_decimals)}', fontsize=12)
            else:
                plt.subplots_adjust(left=0.05, right=0.98, bottom=0.1, top=0.99)
                ax.text(0.5, -0.0, -0.35, s=f'CD={trunc(np.round(cd_cost_l1.item(), decimals=nr_decimals), decs=nr_decimals)}', fontsize=12)
                ax.text(0.5, -0.0, -0.45, s=f'HD={trunc(np.round(hd_p1_to_p2, decimals=nr_decimals), decs=nr_decimals)}', fontsize=12)
                    # verticalalignment='bottom', horizontalalignment='right',
                    # transform=ax.transAxes)
            # ax.text(0.5, 0.5, 0.5, s=f'CD={trunc(cd_cost_l1)}', fontsize=15,
            #         verticalalignment='bottom', horizontalalignment='right',
            #         transform=ax.transAxes)
            # Write HD
            # ax.text(0.5, -0.2, -0.6, s=f'HD={trunc(np.round(hd_p1_to_p2, decimals=nr_decimals), decs=nr_decimals)}', fontsize=12)
            # ax.text(0.5, 0.01, f'CD={trunc(cd_cost_l1)}', fontsize=15)
    
        
            if i < 10:
                it_str = f'00{i}'
            elif i < 100:
                it_str = f'0{i}'
            else:
                it_str = f'{i}'
        
            plt.savefig(f'{out_folder}/{it_str}.png')
            plt.close()
        # if fabric_w_contact:
        #     cd_f_list.append(cd_cost_l1.item())
        #     hd_f_list.append(hd_p1_to_p2)
        # else:
        cd_f_list.append(cd_cost_l1.item())
        hd_f_list.append(hd_p1_to_p2)
        # ax[i, j].set_xticks([])
        # ax[i, j].set_yticks([])
        # plt.gca().axes.get_yaxis().set_visible(False)
        # ax[i,j ].set_axis_off()
    return  np.array(cd_f_list), np.array(hd_f_list)

In [ ]:
source_dir = "/path_to_your_projects/projects/dynamic_dual_manip/results_videos/"
#data_path = "/path_to_your_projects/projects/dynamic_dual_manip/dataset/point_clouds"
data_path = "/path_to_your_projects/datasets/dynamic_cloth_bench"
# primitive_type = "dynamic"
# primitive_type = "quasi_static"

primitives = ['dynamic', 'quasi_static']
# primitives = ['quasi_static']


sim_folders = ['sofa', 'mujoco3', 'softgym', 'bullet']
# sim_folders = ['mujoco3', 'bullet', 'softgym', 'sofa']


all_cloth_types = ["cotton", "chequered", "linen"]
# all_cloth_types = ["chequered"]

# Create the output folders
for sim in sim_folders:
    for cloth_type in all_cloth_types:
        for primitive in primitives:
            out_folder = f"{source_dir}/{sim}_{cloth_type}_{primitive}"
            if not os.path.exists(out_folder):
                os.makedirs(out_folder)
            
            

all_cloth_types = ["cotton", "chequered", "linen"]
# all_cloth_types = ["chequered"]
        
for fd in os.listdir(source_dir):
    if 'cloth_sequence' in fd:
        if all_cloth_types[0] in fd:
            cloth_type = all_cloth_types[0]
            target = "cotton_rag_1"
            # continue
        elif all_cloth_types[1] in fd:
            cloth_type = all_cloth_types[1]
            target = "chequered_rag_1"
            # continue
        else:
            cloth_type = all_cloth_types[2]
            target = "linen_rag_1"
            # continue
        
        if 'quasi_static' in fd:
            primitive_type = "quasi_static"
            # continue
        elif 'dynamic' in fd:
            primitive_type = "dynamic"
            # continue
        
        target_path = f"{data_path}/{target}/{primitive_type}/cloud"
        target_tgt = load_targets(target_path)
        
        
        cloth_sequence = np.load(source_dir+fd)
        
        print(f"Length of tgt={len(target_tgt)}, sequence={cloth_sequence.shape}")
        if 'mujoco3' in fd:
            env_name = 'mujoco3'
            # continue
        elif 'bullet' in fd:
            env_name = 'bullet'
            # continue
        elif 'softgym' in fd:
            env_name = 'softgym'
            # continue
        else:
            env_name = 'sofa'
            # continue
        
        print(f"sim={env_name}, type={cloth_type} np shape={cloth_sequence.shape}")
        
        out_folder = f"{source_dir}/{env_name}_{cloth_type}_{primitive_type}"

        if primitive_type == 'quasi_static':
            elev = 5
            azim = 30
        else:
            elev = 5
            azim = 13

        cd_f_arr, hd_f_arr = plot_and_save_cloth_sequence(cloth_sequence=cloth_sequence,
                                                          sim_name=env_name,
                                                          out_folder=out_folder,
                                                          chequered_type=target,
                                                          elev=elev, azim=azim,
                                                          primitive=primitive_type, plot=True)

        # Save the result to a txt file
        with open(f'{out_folder}/result.txt', 'w') as f:
            f.write(f'CDf mean={trunc(np.round(cd_f_arr.mean(), decimals=nr_decimals), decs=nr_decimals)}'
                    f' var={trunc(np.round(cd_f_arr.std(), decimals=nr_decimals), decs=nr_decimals)}\n'
                    f'HDf mean={trunc(np.round(hd_f_arr.mean(), decimals=nr_decimals), decs=nr_decimals)}'
                    f' var={trunc(np.round(hd_f_arr.std(), decimals=nr_decimals), decs=nr_decimals)}\n'
                    )

    